In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import compress

from datetime import datetime
from dateutil.parser import parse

import math
import os
import copy

import pickle

In [2]:
# Reading the text file
indy_data = pd.read_csv('indy_opioid-geo.csv', engine = 'python')

In [3]:
indy_data.head()

,Case_Number,Year,Sex,Race,DOD,Age,Military,Employment,Education,Marital_Status,...,MPreDirectional,MName,MSuffix,MCity,MState,MZip,QualityInjury,FeatureMatchingHierarchyNotesInjury,LatitudeInjury,LongitudeInjury
0,100064,2010,1,1,1/14/10,20,0.0,2.0,2.0,1,...,NaN,HARLAN,ST,Indianapolis,IN,46203,ExactParcelCentroidPoint,NaN,39.746800,-86.125646
1,100076,2010,1,2,1/16/10,42,0.0,1.0,2.0,1,...,N,DENNY,ST,Indianapolis,IN,46218,ExactParcelCentroidPoint,NaN,39.816930,-86.101735
2,100079,2010,0,1,1/17/10,28,0.0,1.0,4.0,1,...,NaN,STONEROOT,PL,Indianapolis,IN,46203,ExactParcelCentroidPoint,NaN,39.726285,-86.063188
3,100092,2010,1,2,1/20/10,47,0.0,1.0,6.0,2,...,NaN,MURPHYS,LNDG,Indianapolis,IN,46220,USPSZipAreaCentroid,NaN,39.867992,-86.108018
4,100095,2010,1,1,1/21/10,47,0.0,1.0,4.0,2,...,NaN,SHILO,RD,Unionville,IN,47468,AddressRangeInterpolation,NaN,39.651268,-86.167904


In [4]:
# indy_data.iloc[:, 26:206]

In [5]:
drug_test = indy_data.iloc[:, 26:206]

In [6]:
drug_test = drug_test.drop(['Illicits','Drugs_of_Interest', 'Any_Heroin', 'Any_Fentanyl',
                            'Any_Illicit_Opioid', 'Any_Prescription_Opioid', 'Any_Opioid',
                            'Any_Benzodiazepine', 'Any_Cocaine', 'Any_Methamphetamine',
                            'Drug_Concentrations'], axis=1)

In [7]:
drug_test[(drug_test.sum(axis=1)) > 1].head()

,Alcohols,Ethanol,Acetone,Isopropanol,Amphetamines,Amphetamine,Methamphetamine,Ephedrine,MDMA,MDA,...,Fentanyl_Result,Norfentanyl_Result,Oxycodone_Result,Hydrocodone_Result,Oxymorphone_Result,Hydromorphone_Result,Dihydrocodeine_Result,Cocaine_Result,Benzoylecgonine_Result,Methadone_Result
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
def getDrugs(row):
    row_preds = [] #''
    for col_name, value in row.items():
        if value == 1:
            #row_preds += ' ' + col_name
            row_preds.append(col_name)
    return(row_preds)

In [9]:
indy_data['drug_preds'] = drug_test.apply(getDrugs, axis = 1)

In [10]:
indy_data['drug_preds']

0                  [Benzodiazepine, Opioids, Oxymorphone]
1                            [Alcohols, Ethanol, Cocaine]
2                            [Alcohols, Ethanol, Cocaine]
3       [Benzodiazepine, Opioids, Hydrocodone, Hydromo...
4                                               [Cocaine]
5                                                      []
6       [Benzodiazepine, Opioids, Morphine, Hydrocodon...
7                                               [Cocaine]
8                                        [Benzodiazepine]
9                [6_MAM, Opioids, Oxycodone, Hydrocodone]
10      [Alcohols, Ethanol, 6_MAM, Heroin_from_combo, ...
11      [Benzodiazepine, Opioids, Oxycodone, Oxymorphone]
12                                    [Alcohols, Ethanol]
13                                       [Benzodiazepine]
14      [6_MAM, Heroin_from_combo, Opioids, Morphine, ...
15      [Alcohols, Ethanol, 6_MAM, Opioids, Hydromorph...
16                      [Cocaine, Opioids, Hydromorphone]
17      [Alcoh

In [11]:
#drug_test.loc[(drug_test['Fentanyl'] == 1) & (drug_test['Cocaine'] == 1)]

In [12]:
import spacy
from spacy.lang.en import English
parser = English()

In [13]:
# Function to tokenize titles

def tokenize_1(title):
    tokens = parser(title.lower())
    tokens = [token.orth_ for token in tokens if not token.orth_.isspace()]
    return(tokens)

In [14]:
# Apply tokenizer to the titles in the dataframe
#indy_data['tokenized_titles'] = indy_data.iloc[:,[24, 246]].apply(tokenize_1)

In [15]:
import drugLookup

In [16]:
# Load drugs dictionary
drugs_dict = drugLookup.get_drugs_dict()

complete_multi_word_list = []

drug_index = 0
#For loop that gets all the synonyms of drugs with more than one word
for drug in drugs_dict.keys():
    slang_words = drugs_dict[drug]
    multi_word_list = []
    for slang_word in slang_words:
        if ' ' in slang_word:
            multi_word_list.append(slang_word)
    complete_multi_word_list.append(multi_word_list)

In [17]:
# Create multi_word dictionary
drug_cats = list(drugs_dict.keys())
multi_drug_dict = dict(zip(drug_cats, complete_multi_word_list))

In [18]:
# Load dictionary of drugs
drugs_dict = drugLookup.get_drugs_dict()


# Get list of possible drugs for each listing
def categorize_listing(title_set):
    #     Function takes in set of two: first entry is raw_title, second is tokenized_title
    possible_drugs = []
    list_title_set = list(title_set)
    raw_title = list_title_set[0]
    tokenized_title = list_title_set[1]
    for drug in drugs_dict.keys():
        if any(token in drugs_dict[drug] for token in tokenized_title):  # uses tokenization
            possible_drugs.append(drug)
        if any(multi_word in raw_title.lower() for multi_word in multi_drug_dict[drug]):
            possible_drugs.append(drug)
            break
    return possible_drugs

In [19]:
# Categorize each listing
indy_data['drug_prediction'] = indy_data.iloc[:,[24,246]].apply(categorize_listing, axis = 1)

In [20]:
indy_data.iloc[:,246]

0                  [Benzodiazepine, Opioids, Oxymorphone]
1                            [Alcohols, Ethanol, Cocaine]
2                            [Alcohols, Ethanol, Cocaine]
3       [Benzodiazepine, Opioids, Hydrocodone, Hydromo...
4                                               [Cocaine]
5                                                      []
6       [Benzodiazepine, Opioids, Morphine, Hydrocodon...
7                                               [Cocaine]
8                                        [Benzodiazepine]
9                [6_MAM, Opioids, Oxycodone, Hydrocodone]
10      [Alcohols, Ethanol, 6_MAM, Heroin_from_combo, ...
11      [Benzodiazepine, Opioids, Oxycodone, Oxymorphone]
12                                    [Alcohols, Ethanol]
13                                       [Benzodiazepine]
14      [6_MAM, Heroin_from_combo, Opioids, Morphine, ...
15      [Alcohols, Ethanol, 6_MAM, Opioids, Hydromorph...
16                      [Cocaine, Opioids, Hydromorphone]
17      [Alcoh

## Co-Occurance

In [21]:
#Get the list unique drugs for each buyer
drug_series = indy_data['drug_preds'].tolist()

In [22]:
from collections import Counter
from itertools import combinations

counter  = Counter()

In [23]:
#find the most common pairings of each list
for sub in drug_series:
    if len(drug_series) < 2:
        continue
    sub.sort()
    for comb in combinations(sub,2):
        counter[comb] += 1

In [24]:
#Print the most common pairings
most_common = counter.most_common()

most_common

In [25]:
# Read in data (from pickle file)
file = open('market_co','rb')
market_co = pickle.load(file)

In [26]:
market_co.columns = ['drug_co','occurs']

In [27]:
market_co.head()

,drug_co,occurs
0,"(cocaine, marijuana)",256
1,"(marijuana, not_drugs)",246
2,"(, not_drugs)",222
3,"(ecstasy, marijuana)",221
4,"(, marijuana)",199


In [28]:
indy_co = pd.DataFrame(most_common)
indy_co.columns = ['drug_co', 'occurs']

In [29]:
indy_co.head()

,drug_co,occurs
0,"(Morphine, Opioids)",723
1,"(Benzodiazepine, Opioids)",606
2,"(6_MAM, Opioids)",565
3,"(6_MAM, Morphine)",562
4,"(Codeine, Opioids)",533


In [30]:
#tuple(item.lower() for item in indy_co['drug_coi'][1])

In [31]:
def tokenize_2(tup):
    thing = tuple(item.lower() for item in tup)# in indy_co['drug_coi'][i])
    return(thing)

In [32]:
indy_co['drug_co'] = indy_co['drug_co'].apply(tokenize_2)

In [33]:
indy_co

,drug_co,occurs
0,"(morphine, opioids)",723
1,"(benzodiazepine, opioids)",606
2,"(6_mam, opioids)",565
3,"(6_mam, morphine)",562
4,"(codeine, opioids)",533
5,"(6_mam, codeine)",531
6,"(6_mam, heroin_from_combo)",531
7,"(codeine, heroin_from_combo)",531
8,"(codeine, morphine)",531
9,"(heroin_from_combo, morphine)",531


In [34]:
#save as pickle
import pickle
file=open('indy_co' , 'wb')
pickle.dump(indy_co, file)
file.close()

In [36]:
#Export as csv
indy_co.to_csv('indy_co-occurence.csv')